In [ ]:
import numpy as np
from scipy import signal
from scipy import linalg
import control
import matplotlib.pyplot as plt
import pickle

In [ ]:
# file = open('filter_ss.pickle', 'rb')
# [A,B,C,D] = pickle.load(file)
# file.close()
%store -r
[A,B,C,D] = the_filter
print(A,B,C,D)

### Matrix Balancing
Matrix balancing is a preprocessing step in linear algebra computations such as the computation of eigenvalues of a matrix. Such computations are known to be numerically unstable if the matrix is unbalanced, that is the L2 norm of some rows and their corresponding columns are different by orders of magnitude.

https://escholarship.org/uc/item/3847b5dr

In [ ]:
[A, T] = linalg.matrix_balance(A)
B = linalg.solve(T, B) 
C = C @ T

## State Space Model
The shape of the state space equations determine the order of the $\Sigma\Delta$ filter. Below is the continuous time representation

$\dot{x} = A_{c}x + B_{c}u$<br>
$y = C_{c}x + D_{c}u$

The corresponding based $\delta$ model[^fn1].

$\delta$x = $A_{\delta}x + B_{\delta}u$<br>
$y = C_{\delta}x + D_{\delta}u$

### Converting from Continuous Time to Sampled Time

In [ ]:
OSR = 256      # oversample ratio
fb = 22050     # nyquist
fs = OSR*2*fb  # sampling frequency
ts = 1/fs      # sampling period

${\Delta} = ts = \dfrac{1}{fs}$ </p>
$A_{\delta} = \dfrac{\exp(A_{c} - \textit{I})}{\Delta}$ <br>
$B_{\delta} = \dfrac{1}{\Delta}\int_{0}^{\Delta}\exp(A_{c}(t - \tau))B_{c}u(\tau)\;d\tau$ <br>
$C_{\delta} = C_{c}$ <br>
$D_{\delta} = D_{c}$ <br>

In [ ]:
Ad = (linalg.expm(A*ts) - np.eye(A.shape[0])) / ts
Bd = ((linalg.expm(A*ts) - np.eye(A.shape[0]) ) @ B) / ts
Bd = np.linalg.inv(A) @ Bd
Cd = C
Dd = D

### Structural Transformation of Filter

A similarity transform of the sampled time state space matrix is evaluated. This puts the $\delta$DFIIt structure into the observable canonical form, $T_{0}$. $T_{1}$ is first calcuated as below. [^fn3]

$T_{1}=\left[\begin{array}{c}
C_{\delta}\\
C_{\delta}A_{\delta}\\
\vdots\\
C_{\delta}A_{\delta}^{n-1}
\end{array}\right]^{-1}\left[\begin{array}{c}
0\\
0\\
\vdots\\
1
\end{array}\right]$

In [ ]:
e = np.zeros((A.shape[1], 1))
e[-1] = 1
O = control.obsv(A, C)
[U, S, Vh] = linalg.svd(O)
V = Vh.T
S = np.diag(S)
S_inv = linalg.solve(S, np.eye(S.shape[0]))
T_inv = V @ S_inv @ U.conj().T
T1 = T_inv @ e;

$T_{0}=\left[\begin{array}{ccccc}
A_{\delta}^{n-1}T_{1} & A_{\delta}^{n-2}T_{1} & \ldots & A_{\delta}T_{1} & T_{1}\end{array}\right]$

In [ ]:
n  = A.shape[1]
T0 = np.zeros((n,n))

for i in range(1, n+1):
  column = np.power(A, n-i) @ T1
  T0[:, i-1] = column[:,0]

$\tilde{A_{\delta}}=T_{0}^{-1}A_{\delta}T_{0}$ <br>
$\tilde{B_{\delta}}=T_{0}^{-1}B_{\delta}$ <br>
$\tilde{C_{\delta}}=C_{\delta}T_{0}$

In [ ]:
Ad_t = linalg.solve(T0, A @ T0)
Bd_t = linalg.solve(T0, B)
Cd_t = C @ T0;
Dd_t = D;

In [ ]:
[num_t, den_t] = signal.ss2tf(Ad_t,Bd_t,Cd_t,Dd_t)

### $\Sigma\Delta$ Transfer Function (Bode Vectors)

$\large \delta I = ((e^{\frac{-j\omega}{\Delta}} - 1) \times \frac{1}{\Delta})\times I$

$H(\delta)=\tilde{C_{\delta}}(\delta I-\tilde{A_{\delta}})\tilde{B_{\delta}}+D_{\delta}$

In [ ]:
def delta_bode(A,B,C,D,f,ts):
  q     = C.shape[0]
  p     = B.shape[1]
  fs    = 1/ts
  mag   = np.zeros((q,p,f.shape[0]))
  phz   = np.zeros((q,p,f.shape[0]))
  delta = (np.exp(1j*2*np.pi*(f/fs))-1)/ts

  for i in range(f.shape[0]):
      A_d = delta[i] * np.eye(A.shape[0]) - A
        
      [A_d, T_d] = linalg.matrix_balance(A_d)
      B_d = linalg.solve(T_d, B)
      C_d = C @ T_d
      
      h   = linalg.solve(A_d, B_d)
      h   = (C_d @ h) + D
      mag[:,:,i] = np.abs(h)
      phz[:,:,i] = 180*np.arctan2(np.imag(h),np.real(h))/np.pi
  return mag, phz


In [ ]:
f = np.logspace(0,np.log10(fb),2**10)

### Dynamic Range Scaling of State Variable Integrators

$\tilde{A_{\delta}^{'}}=T_{s}^{-1}T_{0}^{-1}A_{\delta}T_{0}T_{s}$ <br>
$\tilde{B_{\delta}^{'}}=T_{s}^{-1}T_{0}^{-1}B_{\delta}$ <br>
$\tilde{C_{\delta}^{'}}=C_{\delta}T_{0}T_{s}$ <br>

$H(\delta)=\tilde{C_{\delta}^{'}}(\delta I-\tilde{A_{\delta}^{'}})\tilde{B_{\delta}^{'}}+D_{\delta}$

#### transfer functions from the input to the ith state integrator
$f_{i}\left(\delta\right)=\frac{x_{i}\left(\delta\right)}{u\left(\delta\right)}$ = $T_{s}^{-1}T_{0}^{-1}\left(\delta I-A_{\delta}\right)^{-1}B_{\delta}$

In [ ]:
T0_inv = linalg.solve(T0, np.eye(T0.shape[0]))
[f_i, phz] = delta_bode(Ad,Bd,T0_inv,0,f,ts)

$\left\Vert H(\frac{e^{j\omega}-1}{\Delta})\right\Vert _{p}=\left[\frac{1}{2\pi}\intop_{-\pi}^{\pi}|H\left(\frac{e^{j\omega}-1}{\Delta}\right)|^{p}d\omega\right]^{1/p}$

$||f(\delta)||_{\infty}	=T_{s}^{-1}||T_{0}^{-1}\left(\delta I-A_{\delta}\right)^{-1}B_{\delta}||_{\infty}
	=\left[\begin{array}{ccc}
1 & \cdots & 1\end{array}\right]^{T}$

In [ ]:
f_norm = np.zeros(Ad.shape[0])

for i in range(f_norm.shape[0]):
  f_norm[i] = linalg.norm(f_i[i], np.inf, axis=1)

$T_{s}=diag\left[k_{1}^{-1},\left(k_{1}k_{2}\right)^{-1},\ldots,\left(k_{1}k_{2}\ldots k_{n}\right)^{-1}\right]$

$\tilde{k}_{i}=\frac{2^{\left\lfloor log_{2}\Delta\cdot k_{i}^{-1}\right\rfloor }}{\Delta}$

In [ ]:
Ts = np.zeros(A.shape)
k = np.zeros(f_norm.shape[0])
k_inv = np.zeros(f_norm.shape[0])

for i in range(f_norm.shape[0]):
  if i == 0:
    k[i] = 1/f_norm[i]
  else:
    k[i] = 1/(np.prod(k[:i])*f_norm[i])

  k_inv[i] = 2**np.floor(np.log2(ts/k[i]))/ts
  Ts[i,i] = np.prod(k_inv[0:i+1])


In [ ]:
num_ts      = np.copy(num_t[0])
num_ts[1:] /= np.diag(Ts)
num_ts[0]   = num_t[0][0]
den_ts      = np.copy(den_t)
den_ts[1:] /= np.diag(Ts)
den_ts[0]   = 1

sdf_beta  = num_ts;
sdf_alpha = den_ts;


In [ ]:
# %store sdf_beta
sdf_beta

In [ ]:
# %store sdf_alpha
sdf_alpha

In [ ]:
# file = open('delta_filter_ss.pickle', 'wb')
# pickle.dump((Ad,Bd,Cd,Dd,T0,Ts,f,ts), file)
# file.close()
the_delta_filter = [Ad,Bd,Cd,Dd,T0,Ts,f,ts]
%store the_delta_filter

[^fn1]: [$\Sigma\Delta$ Stream Computation: A New Paradigm for Low Power and High Resolution Feedback Control](https://escholarship.org/uc/item/4f46n0h6) by Poverelli, Joseph Sam <br>
[^fn2]: [Digital Control and Estimation: A Unified Approach](https://dl.acm.org/doi/10.5555/574885) by Richard H Middleton & 
Graham C Goodwin <br>
[^fn3]: [A generalized direct-form delta operator-based IIR filter with minimum noise gain and sensitivity](https://ieeexplore.ieee.org/document/933811) by Ngai Wong & Tung-Sang Ng <br>